### Default Setting 

In [39]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
    from IPython.display import Markdown, display
from llama_index import QueryBundle 
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaTokenizerFast, BitsAndBytesConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)
import chromadb
import transformers
import pandas as pd
import json 
import openai
import os
import getpass
import torch

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../models')
config_path = os.path.join(default_path, '../../config')

In [3]:
model_dir = os.path.join(model_path, "mistral_origin")

In [4]:
tokenizer = LlamaTokenizerFast.from_pretrained(os.path.join(model_dir, 'tokenizer'))   # LlamaTokenizer (x)  -> LlamaTokenizerFast (o)
model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [6]:
generation_config = GenerationConfig(
    temperature=0.8,
    do_sample=True,
    top_p=0.95,
    max_new_tokens=512,
)

### Retrieve 

In [7]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.vector_stores import SimpleVectorStore
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 
from llama_index.postprocessor import SentenceTransformerRerank 
from llama_index.postprocessor import KeywordNodePostprocessor 
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [8]:
db = chromadb.PersistentClient(path="/rag/db/chroma-llama")
desc_collection = db.get_or_create_collection("desc")
feature_collection = db.get_or_create_collection("feature")
qualification_collection = db.get_or_create_collection("qualification")

In [9]:
desc_store = ChromaVectorStore(chroma_collection=desc_collection)
feature_store = ChromaVectorStore(chroma_collection=feature_collection)
qualification_store = ChromaVectorStore(chroma_collection=qualification_collection)

In [10]:
desc_collection.count()   # 2887

3390

#### - Embedding Model  - kakaobank 

In [11]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [12]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [13]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [14]:
embedding_service = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model, llm=None)

LLM is explicitly disabled. Using MockLLM.


In [15]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = VectorStoreIndex.from_vector_store(feature_store, service_context=embedding_service)
desc_idx = VectorStoreIndex.from_vector_store(desc_store, service_context=embedding_service)
qualification_idx = VectorStoreIndex.from_vector_store(qualification_store, service_context=embedding_service)

#### - Retriever 

In [16]:
retriever = VectorIndexRetriever(
    index = desc_idx,
    service_context=embedding_service,
    similarity_top_k = 10, 
    verbose=True
)

In [17]:
def get_retrieved_nodes(
    retriever, query_str, vector_top_k=10, similarity_cutoff=0.6, reranker_top_n=3, service_context=None, with_reranker=False
):
    # query bundle 생성 
    query_bundle = QueryBundle(query_str)

    # 유사도가 제일 높은 node 추출 
    retrieved_nodes = retriever.retrieve(query_bundle)

    # 전처리  - 유사 점수 기준 Cutoff 
    node_postprocessors = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
    processed_nodes = node_postprocessors.postprocess_nodes(retrieved_nodes)
    
    if with_reranker:   # 재순위화 
        reranker = SentenceTransformerRerank(
            model='bongsoo/albert-small-kor-cross-encoder-v1',
            top_n=reranker_top_n,
        )
        reranked_nodes = reranker.postprocess_nodes(
            processed_nodes, query_bundle
        )
    return reranked_nodes

In [18]:
with_reranker = True
cutoff = 0.2

queries = ['렌탈 요금 청구 할인', '주유소 할인', 'KT 통신 요금 할인', '아파트 관리비', '청년들을 위한 적금 상품 '] 
query = '렌탈 요금 청구 할인' # queries[0]

In [19]:
import time

start = time.time()
retrieved_nodes = retriever.retrieve(query)
nodes = get_retrieved_nodes(retriever, query, similarity_cutoff=cutoff, service_context=embedding_service, with_reranker=with_reranker)
end = time.time() - start 
print(end)

1.677539348602295


In [20]:
nodes[0].node.relationships['1'].metadata

{'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}

In [21]:
for key, value in nodes[0].node.relationships.items():
    print(f"key: {key}, value: {value}", end='\n\n')

key: 1, value: node_id='카드_1064' node_type=<ObjectType.DOCUMENT: '4'> metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'} hash='90392ac21267d9c5d97da6f6c6c1d471c44ff79c0739632924fc94e81f1e6dd5'

key: 2, value: node_id='6d9d34b1-7dee-4404-bf77-7cb6318c9b50' node_type=<ObjectType.TEXT: '1'> metadata={'category': 'card', 'name': 'LG전자 플러스 하나카드'} hash='d51f79921ab96c6dfb2987685bd78bd4904a27558d504495f6ceb9ab01dfc8c7'

key: 3, value: node_id='28b2a799-c2f7-4a95-9a14-ac72587df239' node_type=<ObjectType.TEXT: '1'> metadata={} hash='2255a2418f9a9575fff5e8c4ec130a434b07d8d1be99a42019b79a788460cb2a'



In [22]:
def get_info(retrieved_node):
    '''
    id, text, score 반환 
    '''
    id = retrieved_node.node.relationships['1'].node_id
    name = retrieved_node.node.relationships['1'].metadata['name']
    txt = retrieved_node.node.text 
    score = retrieved_node.score 
    return [id, name, txt, score] 

In [23]:
nodes

[NodeWithScore(node=TextNode(id_='1e4c08b7-b061-4247-a5ff-72288783ab07', embedding=None, metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category', 'name'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='카드_1064', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'card', 'name': 'NEW 렌탈 플러스 하나카드'}, hash='90392ac21267d9c5d97da6f6c6c1d471c44ff79c0739632924fc94e81f1e6dd5'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='6d9d34b1-7dee-4404-bf77-7cb6318c9b50', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'card', 'name': 'LG전자 플러스 하나카드'}, hash='d51f79921ab96c6dfb2987685bd78bd4904a27558d504495f6ceb9ab01dfc8c7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='28b2a799-c2f7-4a95-9a14-ac72587df239', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2255a2418f9a9575fff5e8c4ec130a434b07d8d1be99a42019b79a788460cb2a')}, hash='1221d93222d3aabae38c84ccbe1a536bf41

In [24]:
print([node.text for node in retrieved_nodes])

['렌탈요금 자동이체 시 13,000원 청구할인', '렌탈요금 자동이체 시 13,000원 청구할인', '렌탈 자동이체 시 월 8,000원 또는 월 13,000원 할인', '렌탈 자동이체 시 월 8,000원 또는 월 13,000원 할인', '청호나이스 렌탈요금 자동이체하고 최대 2만원 할인받자!', '청호나이스 렌탈요금 자동이체하고 최대 2만원 할인받자!', '＇SK매직 삼성카드＇는 SK매직 상품 구매시 24·36개월 무이자 할부 혜택이 아닌 렌탈료 할인 혜택이 제공되는 카드입니다', '＇SK매직 삼성카드＇는 SK매직 상품 구매시 24·36개월 무이자 할부 혜택이 아닌 렌탈료 할인 혜택이 제공되는 카드입니다', '렌탈료 및 장기할부 이용 시 최대 20,000원 할인!', '렌탈료 및 장기할부 이용 시 최대 20,000원 할인!']


In [25]:
contexts = [node.text for node in nodes]
scores = [node.score for node in nodes]
contexts, scores 

(['렌탈요금 자동이체 시 13,000원 청구할인',
  '렌탈요금 자동이체 시 13,000원 청구할인',
  '청호나이스 렌탈요금 자동이체하고 최대 2만원 할인받자!'],
 [0.65832305, 0.65832305, 0.45793614])

In [26]:
get_info(nodes[0])

['카드_1064', 'NEW 렌탈 플러스 하나카드', '렌탈요금 자동이체 시 13,000원 청구할인', 0.65832305]

In [27]:
info = get_info(nodes[0])
info

['카드_1064', 'NEW 렌탈 플러스 하나카드', '렌탈요금 자동이체 시 13,000원 청구할인', 0.65832305]

In [28]:
product = info[1]
context = f"'{info[1]}'은 {info[2]} 관련 상품입니다."
product, context

('NEW 렌탈 플러스 하나카드', "'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다.")

### Augment 

In [29]:
prompt_template = (
    f"""<s>[INST] 질문: {query} \n
    관련 정보: {context} \n 
    관련 정보를 바탕으로 질문에 답해줘 [/INST] \n"""
)

print(prompt_template)

<s>[INST] 질문: 렌탈 요금 청구 할인 

    관련 정보: 'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다. 
 
    관련 정보를 바탕으로 질문에 답해줘 [/INST] 



### Generation 

In [30]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [31]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [32]:
gened = model.generate(
    **tokenizer(
        prompt_template,
        return_tensors='pt',
        return_token_type_ids=False
    ).to('cuda'),
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    # streamer=streamer,
)

In [33]:
result_str = tokenizer.decode(gened[0])

start_tag = f"[/INST]"
start_index = result_str.find(start_tag)

if start_index != -1:
    result_str = result_str[start_index + len(start_tag):].strip()

In [34]:
print(result_str)

NEW 렌탈 플러스 하나카드의 경우, 렌탈요금 자동이체 시 13,000원 청구할인 상품입니다. 이는 렌탈 요금의 지불 편의를 위해 자동 이체 방식을 선택하면 추가 할인 혜택을 제공하는 상품입니다. 따라서 렌탈 요금 자동이체 시 13,000원 청구할인이 적용됩니다. 그러나 렌탈 요금의 유형에 따라 다른 할인 혜택이 적용될 수 있으므로, 상품 설명서나 고객센터에 문의하여 세부적인 정보를 확인해야 합니다.</s>


### LangChain

In [35]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaTokenizerFast, BitsAndBytesConfig
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain_community.llms import LlamaCpp

In [36]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512

In [72]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [58]:
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=1,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=512,
    device=0
)

In [63]:
prompt_template

"<s>[INST] 질문: 렌탈 요금 청구 할인 \n\n    관련 정보: 'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다. \n \n    관련 정보를 바탕으로 질문에 답해줘 [/INST] \n"

In [59]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [87]:
prompt_template = (
    f"""
    ### <s> [INST]
    참고: 다음 질문에 대해 너의 금융 정보에 기반해서 답을 해줘. 참고할만한 정보는 다음과 같아. 
    
    {context}

    ### Question:
    {query}
    [/INST] """
)

print(prompt_template)


    ### <s> [INST]
    참고: 다음 질문에 대해 너의 금융 정보에 기반해서 답을 해줘. 참고할만한 정보는 다음과 같아. 
    
    'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다.

    ### Question:
    렌탈 요금 청구 할인
    [/INST] 


In [88]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=prompt_template,
)
# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [89]:
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=[], template="\n    ### <s> [INST]\n    참고: 다음 질문에 대해 너의 금융 정보에 기반해서 답을 해줘. 참고할만한 정보는 다음과 같아. \n    \n    'NEW 렌탈 플러스 하나카드'은 렌탈요금 자동이체 시 13,000원 청구할인 관련 상품입니다.\n\n    ### Question:\n    렌탈 요금 청구 할인\n    [/INST] "), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fbafab73be0>))

In [90]:
print(llm_chain.invoke({"context": "", "query": query}))

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'context': '', 'query': '렌탈 요금 청구 할인', 'text': '렌탈 요금 청구 할인은 렌탈 회사가 고객에게 제공하는 서비스로, 일정한 금액을 매월 자동으로 청구하지만 그 금액을 일부 할인해주는 것입니다. 이를 통해 고객은 더 저렴한 금액으로 장비를 사용할 수 있습니다. \n\nNEW 렌탈 플러스 하나카드는 렌탈 요금 자동이체 시 13,000원 청구할인 관련 상품입니다. 따라서 이 상품을 선택하면 렌탈 요금을 13,000원 할인받을 수 있습니다. \n\n하지만 렌탈 요금 청구 할인은 일부 제한이 있을 수 있으므로 상품 설명서를 꼭 확인하고 조건에 맞게 활용해야 합니다.'}


In [ ]:
### LlamaIndexRetriever <-> Langchain 

In [102]:
desc_engine = desc_idx.as_retriever(retriever_mode="llm")